In [1]:
import numpy as np
import cv2 as cv
import time
#define upper and lower thresholds for white color
white_lower = np.array([0, 0, 180])
white_upper = np.array([180, 30, 255])


In [2]:
url="http://192.168.1.102:4747/video" # ip camera address
cap=cv.VideoCapture(url) #if you using an webcam put 0 as methods input
if not cap.isOpened():
    print("Error: can not open the camera")
    exit()
    
while True:
    start_time = time.perf_counter()
    ret,frame=cap.read()
    if not ret:
        print("Error: can not receive frame. exiting...")
        break
    kernel=(5,5)
    blur=cv.GaussianBlur(frame,kernel,0)  
    hsv = cv.cvtColor(blur, cv.COLOR_BGR2HSV)
    mask_white = cv.inRange(hsv, white_lower, white_upper)
    mask_white = cv.morphologyEx(mask_white, cv.MORPH_CLOSE, kernel)
    mask_white = cv.morphologyEx(mask_white, cv.MORPH_OPEN, kernel)  
    contours, hierarchy=cv.findContours(mask_white,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    for contour in contours:
        area=cv.contourArea(contour)
        if area > 300 : # change to detect bigger or samller objects in white color
            cv.drawContours(frame,contour,-1,(0,255,0),3)
    font = cv.FONT_HERSHEY_SIMPLEX
    end_time = time.perf_counter()
    elapsed = end_time - start_time
    fps_text = f" Loop time: {elapsed*1000:.1f} ms"
    text="total count: "+str(len(contours))+fps_text
    cv.putText(frame,text,(50,50),font,1,(0,255,0),2)
    cv.imshow("video",frame)   
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()
    